This is to demonstrate how to download files given the name of the deployment (e.g. ```test-dem-95-sim```).

In [ ]:
import hyp3_sdk
import os

# Define job-name
job_name = 'test-dem-95-sim'

In [ ]:
# uses .netrc; add `prompt=True` to prompt for credentials; 
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-isce.asf.alaska.edu/')
jobs = hyp3_isce.find_jobs(name=job_name)

job_ids = [job.job_id for job in jobs]

import json
from pathlib import Path
Path(job_name+'json').write_text(json.dumps(job_ids))

In [ ]:
# Generate json file that lists the datasets to be downloaded
df = str(Path(job_name+'json').absolute())

# Save directory
savedir = os.path.join(os.path.abspath(os.getcwd()),'deployment')
if not os.path.exists(savedir):
    os.mkdir(savedir)

In [ ]:
import hyp3_sdk
import json
from pathlib import Path
import concurrent.futures
from tqdm import tqdm

# uses .netrc; add `prompt=True` to prompt for credentials; 
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-isce.asf.alaska.edu/')
job_ids = json.loads(Path(df).read_text())

jobs = hyp3_sdk.Batch()
for ii in job_ids:
    jobs += hyp3_isce.get_job_by_id(ii)

In [ ]:
# Download files in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(tqdm(executor.map(lambda job: job.download_files(savedir), jobs), total=len(jobs)))